### 복습 
- 가전 폴더 안에 모든 데이터파일을 로드해서 하나의 데이터프레임으로 생성 
- 감정에 대한 데이터들이 3개 분류 -> 2개의 분류로 변경 (부정, 중립 -> 부정)
- 감정 데이터가 없는 데이터들은 따로 저장 
- train, test의 비율은 8:2
- Dataset을 기존의 Dataset 구성과 같이 작업
- RawText 데이터를 이용하여 감정분석 모델을 생성 
- SBERT 모델을 이용하여 임베딩 
- 다중퍼셉트론의 모델을 이용하여 감정 분석 (Linear -> ReLU -> DropOut -> Linear)
- 검증 데이터를 이용하여 정확도와 f1_score 확인 
- 감정 데이터가 없는 RawText에서 sample(10)를 출력하여 감정 예측

- 다중퍼셉트론 모델이 아닌 머신러닝 모델 (SVC)을 이용하여 감정 분석 예측

In [1]:
import os 
import re
import pandas as pd 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sentence_transformers import SentenceTransformer

c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 문자 정규화 함수 정의 
def normailize(text):
    text = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [3]:
# 파일의 목록을 로드 -> 목록을 기준으로 데이터를 로드 -> 단순 결합 
file_path = "../data/가전/"
file_list = os.listdir(file_path)
file_list

['3-1.영상음향가전(76).json',
 '3-1.영상음향가전(77).json',
 '3-1.영상음향가전(78).json',
 '3-1.영상음향가전(79).json',
 '3-1.영상음향가전(80).json',
 '3-1.영상음향가전(81).json',
 '3-1.영상음향가전(82).json',
 '3-1.영상음향가전(83).json',
 '3-1.영상음향가전(84).json',
 '3-1.영상음향가전(85).json',
 '3-1.영상음향가전(86).json',
 '3-1.영상음향가전(87).json',
 '3-1.영상음향가전(88).json',
 '3-2.생활미용욕실가전(128).json',
 '3-2.생활미용욕실가전(129).json',
 '3-2.생활미용욕실가전(130).json',
 '3-2.생활미용욕실가전(131).json',
 '3-2.생활미용욕실가전(132).json',
 '3-2.생활미용욕실가전(133).json',
 '3-2.생활미용욕실가전(134).json',
 '3-2.생활미용욕실가전(135).json',
 '3-2.생활미용욕실가전(136).json',
 '3-2.생활미용욕실가전(137).json',
 '3-2.생활미용욕실가전(138).json',
 '3-2.생활미용욕실가전(139).json',
 '3-2.생활미용욕실가전(140).json',
 '3-3.주방가전(127).json',
 '3-3.주방가전(128).json',
 '3-3.주방가전(129).json',
 '3-3.주방가전(130).json',
 '3-3.주방가전(131).json',
 '3-3.주방가전(132).json',
 '3-3.주방가전(133).json',
 '3-3.주방가전(134).json',
 '3-3.주방가전(135).json',
 '3-3.주방가전(136).json',
 '3-3.주방가전(137).json',
 '3-3.주방가전(138).json',
 '3-3.주방가전(139).json',
 '3-4.계절가전(126).json',
 '3-4.계절가전(127)

In [4]:
# 로드한 데이터프레임을 누적으로 결합하기 위해 빈 데이터프레임 생성 
df = pd.DataFrame()

for file in file_list:
    # file : 파일명 
    data = pd.read_json(file_path + file)
    # df에 단순 행 결합 -> df에 다시 대입 
    df = pd.concat( [df, data], axis= 0  )
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4056 entries, 0 to 99
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            4056 non-null   int64  
 1   RawText          4056 non-null   object 
 2   Source           4056 non-null   object 
 3   Domain           4056 non-null   object 
 4   MainCategory     4056 non-null   object 
 5   ProductName      4056 non-null   object 
 6   ReviewScore      4056 non-null   int64  
 7   Syllable         4056 non-null   int64  
 8   Word             4056 non-null   int64  
 9   RDate            4056 non-null   int64  
 10  GeneralPolarity  3678 non-null   float64
 11  Aspects          4056 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 411.9+ KB


In [5]:
# 필요한 컬럼을 제외하고 나머지 컬럼을 무시 
df = df[['RawText', 'GeneralPolarity']]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4056 entries, 0 to 99
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RawText          4056 non-null   object 
 1   GeneralPolarity  3678 non-null   float64
dtypes: float64(1), object(1)
memory usage: 95.1+ KB


In [7]:
df.rename(columns = {
    'GeneralPolarity' : 'label'
}, inplace=True)

In [8]:
# RawText의 문자 정규화 사용
df['RawText'] = df['RawText'].map(normailize)

In [9]:
# RawText에서 길이가 1 이하인 데이터를 제외
df = df.loc[df['RawText'].str.len() > 1]

In [10]:
# 혹시 RawText에 중복 데이터가 존재할수 있으니 중복 제거 
df.drop_duplicates(subset='RawText', inplace=True)

In [11]:
# label이 결측치인 데이터는 따로 저장 
na_df = df.loc[df['label'].isna(), ]
na_df

,RawText,label
13,귀에서 자꾸 빠져요.귀에 꼽는재 질이 미끄러운 재질이라 작은 소품이지만 재질만 바꾸...,NaN
43,아이를 출산한 기념으로 TV를 바꿨습니다. 그전에 쓰던 TV가 꽤나 무거워서 떨어지...,NaN
44,화면에 노이즈가 생깁니다. 저희 가족 중에 아무도 TV 화면을 건드리거나 하지 않았...,NaN
55,이번에 이사하면서 우리 따님께서 방에 TV가 있으면 좋겠다고 하여 방에서 사용할 T...,NaN
93,기존에 사용하던 무선이어폰이 오래되어서 배터리가 광탈하는 바람에 새로운 상품이 필요...,NaN
...,...,...
41,대용량이고 세척이 간편하다는 얘기에 구매를 했는데 저는 별로인 거 같아요...생각했...,NaN
44,요거 진짜 진짜 물건입니다. 처음엔 디자인이 너무 귀여워서 주문하게 되었는데요. 작...,NaN
68,지인의 추천으로 믿고 바로 구매를 해서 현재도 사용중입니다좀 더 많은 분들에게 도움...,NaN
76,다른 에어쿨러와 다르게 슬림한 디자인이 마음에 들어요.심플한 디자인 덕분에 집안 어...,NaN


In [12]:
# df에는 결측치를 제외
df = df.loc[ ~df['label'].isna(), ]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3678 entries, 0 to 99
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   RawText  3678 non-null   object 
 1   label    3678 non-null   float64
dtypes: float64(1), object(1)
memory usage: 86.2+ KB


In [13]:
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3678 entries, 0 to 3677
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   RawText  3678 non-null   object 
 1   label    3678 non-null   float64
dtypes: float64(1), object(1)
memory usage: 57.6+ KB


In [14]:
# label의 데이터의 빈도수를 확인 
df['label'].value_counts()

label
 1.0    2220
 0.0     944
-1.0     514
Name: count, dtype: int64

In [15]:
# label의 데이터들을 int형태로 변경 
df['label'] = df['label'].astype(int)

In [16]:
df['label'].value_counts()

label
 1    2220
 0     944
-1     514
Name: count, dtype: int64

In [17]:
# 삼중 분류의 class를 이진 분류로 변경하기 위해 -1을 0으로 변경 
df['label'] = df['label'].map(
    {
        -1 : 0, 
        0 : 1, 
        1 : 2
    }
)

In [18]:
df['label'].value_counts()

label
2    2220
1     944
0     514
Name: count, dtype: int64

In [19]:
# train, test 로 분할
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label']
)

In [20]:
train_df['label'].value_counts()

label
2    1776
1     755
0     411
Name: count, dtype: int64

In [21]:
model_name = 'BM-K/KoSimCSE-roberta-multitask'
sbert = SentenceTransformer(model_name)

No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.


In [22]:
# 최대 토큰 길이 제한 
sbert.max_seq_length = 128

In [23]:
class SBERTHead(Dataset):
    def __init__(self, texts, labels):
        with torch.inference_mode():
            self.emb = sbert.encode(
                texts, convert_to_tensor=True, normalize_embeddings=True
            )
            self.labels = torch.tensor(labels, dtype=torch.long)
        self.texts = texts
        self.labels2 = labels
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.emb[idx], self.labels[idx]
        # getitem에서 임베딩 처리 
        # res_emb = sbert.encode(
        #     self.texts[idx], convert_to_tensor=True, normalize_embeddings=True
        # )
        # res_label = torch.tensor(self.labels2[idx], dtype=torch.long)
        # return res_emb, res_label

In [24]:
train_ds = SBERTHead(train_df['RawText'].tolist(), train_df['label'].tolist())
test_ds = SBERTHead(test_df['RawText'].tolist(), test_df['label'].tolist())

train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=256)

In [25]:
# 다중 퍼셉트론 구조의 분류 모델 생성 
class MLPHead(nn.Module):
    def __init__(self, input_dim, hidden = 256, num_classes = 2, dropout=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(), 
            nn.Dropout(dropout), 
            nn.Linear(hidden, num_classes)
        )
    def forward(self, x):
        result = self.net(x)
        return result

In [53]:
# sbert 모델에서 출력 차원의 개수를 확인
in_dim = sbert.get_sentence_embedding_dimension()
clf = MLPHead(in_dim, num_classes= 3, dropout=0.3)
# 손실 함수 -> 데이터의 불균혈 문제는 해결하기 위해 class 별 가중치 변화를 각각 부여 
# [0, 1, 2] -> 적을 데이터일수록 높은 변화 
# 5.0 ~ 10.0 : 데이터의 비율이 굉장히 적은 경우
# 2.0 ~ 4.0 : 데이터 비율이 적은 경우
# 1.0 : 가장 많은 데이터비율
crit = nn.CrossEntropyLoss(weight=torch.tensor([5.0, 3.0, 1.0]))
# AdamW -> Adam 개선판
opt = optim.AdamW(clf.parameters(), lr=2e-4)

In [54]:
clf.train()

for epoch in range(10):
    total = 0.0
    for x, y in train_dl:
        opt.zero_grad()
        logits = clf(x)
        loss = crit(logits, y)
        loss.backward()
        opt.step()
        total += loss.item() * x.size(0)
    print(f"epoch : {epoch}, loss : {round(total/len(train_df), 4)}")

epoch : 0, loss : 1.083
epoch : 1, loss : 1.0435
epoch : 2, loss : 0.9921
epoch : 3, loss : 0.935
epoch : 4, loss : 0.8782
epoch : 5, loss : 0.8244
epoch : 6, loss : 0.7853
epoch : 7, loss : 0.7565
epoch : 8, loss : 0.7348
epoch : 9, loss : 0.7178


In [55]:
# 정확도, f1-score 확인 
clf.eval()

y_true , y_pred = [], []

with torch.inference_mode():
    for x, y in test_dl:
        logits = clf(x)
        pred = logits.argmax(dim = 1).tolist()
        y_true.extend(y.tolist())
        y_pred += pred


In [56]:

print('accuracy_score', accuracy_score(y_true, y_pred))
print('f1_score', f1_score(y_true, y_pred, average='macro'))

accuracy_score 0.65625
f1_score 0.662902426647526


In [57]:
samples = na_df['RawText'].sample(15).tolist()
samples

['고장이 난 줄 알고 처음에는 놀랐는데 알고 보니까 제가 전원 버튼 켜는 법을 잘못 숙지했던 거였더라구요 사용법이 잘 적혀있었으면 좋았을 텐데 사용설명서가 자세하지 않아서 별로네요 그래도 금방 바른 길을 찾아서 다행이었습니다 그래서 크게 신경 안 쓸 수 있었네요 그리고 일단 소리 자체가 너무 맑고 깨끗하게 들려서 엘피 들을 맛이 나는 것 같아요 동생도 듣더니 자기도 엘피판을 사와서 이 턴테이블로 듣고 싶다고 난리난리 제가 덕분에 영업까지 시켰답니다 성능이 아주 만족스러운 아이였어요 사용설명서만 딱 챙겨주셨다면 정말 완벽한 제품이었을 겁니다 하지만 지금으로도 너무 굿이에용',
 '건강 쥬스 만들어 달라는 남편을 위해 하나 장만해 봅니다.초고속으로 윙 갈아 준다기에 구매했습니다.그런데 말입니다. 잘 갈리는것 같지는 않아요. 수분이 많은건 그나마 갈아 주는데 수분량이 부족하면 잘 갈리지 않고 건더기가 많이 남아요. . 목 넘김이 부드럽지 못해 잘 안 써져 마음에 안들어요 휴대가 가능한 텀블러 용기라서 사용자 편의를 엄청 생각해 준건 인정합니다.윙 갈아서 다른용기에 담지 않고 뚜껑만 닫아 쓸수있어서 편리해요. 내가 먹을건 아니니까 가끔 과일도 갈아 주고 야채도 갈아 줍니다.사이즈가 아담해서 보관하기 좋아요. 대형이라면 안샀을거예요.',
 '시골이 경기도 전원 주택이라 대식구 가족들 모이면 주로 거실에 신문지 펴놓고 고기를 구워 먹는데요..가스 버너는 아무래도 아이들과 함께 둘러 앉아서 쓰기에는 위험해서 전기 그릴 사봤습니다..우선 전기그릴 사이즈가 대용량이라 많은 고기를 한꺼번에 구울 수 있어서 좋아요..또 가스 버너처럼 어느 일부분만 열이 가해지는게 아니라 그릴 전체가 전기로 통일성있게 달구워져서 그 부분이 만족스러워요.다이얼 조작으로 원하는 온도를 손쉽게 조절할 수 있어서 고기 구울때 편하더라구요..그릴 밑 바닥이 다이아몬드 코팅이 되어 있어서 별도로 기름을 두르지 않아도 눌러 붙지 않고 고기에서 나오는 지방 기름만으로도 충분히 맛있게 구워져서 좋아요.. 고기가

In [58]:
@torch.no_grad()
def predict_review(texts, batch_size = 128):
    if isinstance(texts, str):
        texts = [texts]
    texts_norm = [normailize(t) for t in texts]  

    sbert.eval()
    clf.eval()

    result = []

    for idx in range(0, len(texts), batch_size):
        batch_text = texts_norm[idx : idx + batch_size]
        embs = sbert.encode(
            batch_text, convert_to_tensor = True, 
            normalize_embeddings = True
        )

        logits = clf(embs)
        probs = logits.softmax(dim = -1)
        preds = probs.argmax(dim = -1).tolist()
        id2label = {
            0 : '부정', 
            1 : '중립', 
            2 : '긍정'
        }
        for idx2, pred in enumerate(preds):
            prob = float(probs[idx2, pred])
            review = texts[idx + idx2]
            label = id2label[pred]
            result.append(
                {
                    'text' : review, 
                    'prob' : prob, 
                    'label' : label
                }
            )
    return result

In [59]:
predict_review(samples)

[{'text': '고장이 난 줄 알고 처음에는 놀랐는데 알고 보니까 제가 전원 버튼 켜는 법을 잘못 숙지했던 거였더라구요 사용법이 잘 적혀있었으면 좋았을 텐데 사용설명서가 자세하지 않아서 별로네요 그래도 금방 바른 길을 찾아서 다행이었습니다 그래서 크게 신경 안 쓸 수 있었네요 그리고 일단 소리 자체가 너무 맑고 깨끗하게 들려서 엘피 들을 맛이 나는 것 같아요 동생도 듣더니 자기도 엘피판을 사와서 이 턴테이블로 듣고 싶다고 난리난리 제가 덕분에 영업까지 시켰답니다 성능이 아주 만족스러운 아이였어요 사용설명서만 딱 챙겨주셨다면 정말 완벽한 제품이었을 겁니다 하지만 지금으로도 너무 굿이에용',
  'prob': 0.5166657567024231,
  'label': '중립'},
 {'text': '건강 쥬스 만들어 달라는 남편을 위해 하나 장만해 봅니다.초고속으로 윙 갈아 준다기에 구매했습니다.그런데 말입니다. 잘 갈리는것 같지는 않아요. 수분이 많은건 그나마 갈아 주는데 수분량이 부족하면 잘 갈리지 않고 건더기가 많이 남아요. . 목 넘김이 부드럽지 못해 잘 안 써져 마음에 안들어요 휴대가 가능한 텀블러 용기라서 사용자 편의를 엄청 생각해 준건 인정합니다.윙 갈아서 다른용기에 담지 않고 뚜껑만 닫아 쓸수있어서 편리해요. 내가 먹을건 아니니까 가끔 과일도 갈아 주고 야채도 갈아 줍니다.사이즈가 아담해서 보관하기 좋아요. 대형이라면 안샀을거예요.',
  'prob': 0.5212316513061523,
  'label': '중립'},
 {'text': '시골이 경기도 전원 주택이라 대식구 가족들 모이면 주로 거실에 신문지 펴놓고 고기를 구워 먹는데요..가스 버너는 아무래도 아이들과 함께 둘러 앉아서 쓰기에는 위험해서 전기 그릴 사봤습니다..우선 전기그릴 사이즈가 대용량이라 많은 고기를 한꺼번에 구울 수 있어서 좋아요..또 가스 버너처럼 어느 일부분만 열이 가해지는게 아니라 그릴 전체가 전기로 통일성있게 달구워져서 그 부분이 만족스러워요.다이얼 조작으로 원하는 온도